In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv('./data/train.csv')

In [3]:
eval = pd.read_csv('./data/eval.csv')

In [4]:
test = pd.read_csv('./data/test.csv')

In [5]:
test.head(5)

,item_cnt_day,item_price,shop_id,item_id
0,1.0,399.0,2,31
1,3.0,300.0,2,486
2,1.0,420.0,2,787
3,1.0,3300.0,2,794
4,1.0,58.0,2,968


In [9]:
eval.head(5)

,item_cnt_day,item_price,shop_id,item_id
0,1.0,399.0,2,31
1,3.0,300.0,2,486
2,1.0,420.0,2,787
3,1.0,3300.0,2,794
4,1.0,58.0,2,968


In [11]:
train.head(5)

,item_cnt_day,item_price,shop_id,item_id,block_31_lag_1_item_price,block_31_lag_1_item_cnt_day,block_30_lag_2_item_cnt_day,block_30_lag_2_item_price,block_29_lag_3_item_cnt_day,block_29_lag_3_item_price,...,block_1_lag_31_item_price,block_1_lag_31_item_cnt_day,block_0_lag_32_item_cnt_day,block_0_lag_32_item_price,block_-1_lag_33_item_cnt_day,block_-1_lag_33_item_price,last_buy,average_interval,avg_buy_rate,std_buy_rate
0,1.0,199.0,2,33,0.0,0.0,1.0,199.0,0.0,0.0,...,0.0,0.0,1.0,499.0,0.0,0.0,2,3.555556,0.323529,0.534880
1,1.0,300.0,2,486,0.0,0.0,0.0,0.0,2.0,300.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3,1.714286,0.382353,0.779071
2,1.0,979.0,2,792,979.0,2.0,0.0,0.0,1.0,979.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1,1.555556,0.411765,0.743360
3,1.0,349.0,2,975,0.0,0.0,1.0,349.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2,2.000000,0.058824,0.238833
4,1.0,499.0,2,1090,0.0,0.0,0.0,0.0,1.0,499.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3,3.000000,0.058824,0.238833


In [13]:
sales = pd.read_csv('./data/sales_train.csv.gz')

In [73]:
def get_label(sales, lag=1):
    max_num = sales['date_block_num'].max()
    curr = max_num - lag + 1
    labels = sales[sales['date_block_num'] == curr]['item_cnt_day'].values
    return labels

In [47]:
def create_data_lag(sales, lag=1):    
    max_num = sales['date_block_num'].max()
    curr = max_num - lag
    print(max_num, curr)
    last_sales =  sales[sales['date_block_num'] == curr]
    last_sales = last_sales.groupby(['shop_id','item_id']).agg({"item_price":'mean',"item_cnt_day":"sum"})
    last_sales['shop_id'] = [i[0] for i in last_sales.index]
    last_sales['item_id'] = [i[1] for i in last_sales.index]
    
    col_item_price = 'block_%s_lag_%s_item_price' % (curr, 0)
    col_item_cnt_day = 'block_%s_lag_%s_item_cnt_day' % (curr, 0)
    
    last_sales[col_item_price] = last_sales['item_price']
    last_sales[col_item_cnt_day] = last_sales['item_cnt_day']
    del last_sales['item_price']
    del last_sales['item_cnt_day']


    for i in range(1,max_num):
        previous_sales = sales.copy()
        previous_sales = previous_sales[['date_block_num', 'shop_id', 'item_id', 'item_price','item_cnt_day']]
        previous_sales = previous_sales[previous_sales['date_block_num'] == curr - 1]
        col_item_price = 'block_%s_lag_%s_item_price' % (curr-1, i)
        col_item_cnt_day = 'block_%s_lag_%s_item_cnt_day' % (curr-1, i)
        previous_sales[col_item_price] = previous_sales['item_price']
        previous_sales[col_item_cnt_day] = previous_sales['item_cnt_day']

        previous_sales= previous_sales.groupby(['shop_id','item_id']).agg({col_item_price:'mean', col_item_cnt_day:"sum"})
        previous_sales['shop_id'] = [i[0] for i in previous_sales.index]
        previous_sales['item_id'] = [i[1] for i in previous_sales.index]

        last_sales = pd.merge(last_sales, previous_sales, on=['shop_id', 'item_id'], how='left')
        curr = curr - 1
    cols = []
    for col in last_sales.columns:
        if 'tuple' in str(type(col)):
            cols.append(col[0]+"_"+col[1])
        else:
            cols.append(col)
    last_sales.columns = cols
    last_sale_clear_nan = last_sales.fillna(0)
    return last_sale_clear_nan

In [75]:
eval_feature = create_data_lag(sales, lag=1)
eval_label = get_label(sales, lag=1)

33 32


C:\Users\17001970\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: 'shop_id' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  """Entry point for launching an IPython kernel.
C:\Users\17001970\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: 'item_id' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  """Entry point for launching an IPython kernel.


In [74]:
test_feature = create_data_lag(sales, lag=2)
test_label = get_label(sales, lag=2)

33 31


C:\Users\17001970\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: 'shop_id' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  """Entry point for launching an IPython kernel.
C:\Users\17001970\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: 'item_id' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  """Entry point for launching an IPython kernel.


In [76]:
train_feature = create_data_lag(sales, lag=3)
train_label = get_label(sales, lag=3)

33 30


C:\Users\17001970\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: 'shop_id' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  """Entry point for launching an IPython kernel.
C:\Users\17001970\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: 'item_id' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  """Entry point for launching an IPython kernel.


# Feature Engineering

In [68]:
def get_column_buy_count(sales):
    col_cnt = []
    for c in sales.columns:
        if 'item_cnt_day' in c:
            col_cnt.append(c)
    return col_cnt

def get_last_buy(sales):
    col_cnt = get_column_buy_count(sales)
    last_buy = []
        
    for items in sales[col_cnt].values:
        month = -1
        for idx,row in enumerate(items):
            if idx > 0 and row > 0:
                if month == -1:
                    month = idx
                    break
        last_buy.append(month)
    sales['last_buy'] = last_buy
    return sales

def get_average_interval(sales):
    col_cnt = get_column_buy_count(sales)
    average_interval = []
    for items in sales[col_cnt].values:
        month_interval = []
        last_buy_month = 0
        for idx,row in enumerate(items):
            if idx > 0 and row > 0:
                month_interval.append(idx - last_buy_month)
                last_buy_month = idx
        average_interval.append(np.mean(month_interval))
    sales['average_interval'] = average_interval
    return sales

def get_avg_buy_rate(sales):  
    col_cnt = get_column_buy_count(sales)
    sales['avg_buy_rate'] = sales[col_cnt].mean(axis=1)
    return sales

def get_std_buy_rate(sales):
    col_cnt = get_column_buy_count(sales)
    sales['std_buy_rate'] = sales[col_cnt].std(axis=1)
    return sales

def generate_feature(last_sale_clear_nan):
    last_sale_clear_nan = get_last_buy(last_sale_clear_nan)
    last_sale_clear_nan = get_average_interval(last_sale_clear_nan)
    last_sale_clear_nan = get_avg_buy_rate(last_sale_clear_nan)
    last_sale_clear_nan = get_std_buy_rate(last_sale_clear_nan)
    return last_sale_clear_nan

# Apply the feature

In [69]:
train_feature = generate_feature(train_feature)

C:\Users\17001970\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\17001970\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [70]:
eval_feature = generate_feature(eval_feature)

C:\Users\17001970\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\17001970\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [71]:
test_feature = generate_feature(test_feature)

C:\Users\17001970\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\17001970\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
